<a href="https://colab.research.google.com/github/floranuta/Data_Circle/blob/Tetiana/notebooks/Task12_DataCleaningPreprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/drive')
# Path of the file to read
#csv_file_path = "D:/REDI/Data_Circle/data/training_set_values.csv"
csv_file_path = "/content/drive/MyDrive/Colab Notebooks/training_set_values.csv"
# Fill in the line below to read the file into a variable home_data
pump_data = pd.read_csv(csv_file_path)


Mounted at /content/drive


In [7]:
# Overview
def missing_report(df):
    missing_count = df.isnull().sum()
    missing_percent = 100 * df.isnull().mean()
    missing_table = (
        pd.DataFrame({
            "MissingCount": missing_count,
            "MissingPercent": missing_percent
        })
        .query("MissingCount > 0")  # оставить только с пропусками
        .sort_values(by="MissingCount", ascending=False)
    )
    return missing_table

missing_table = missing_report(pump_data)
print(missing_table.to_string())

                   MissingCount  MissingPercent
scheme_name               28810       48.501684
scheme_management          3878        6.528620
installer                  3655        6.153199
funder                     3637        6.122896
public_meeting             3334        5.612795
permit                     3056        5.144781
subvillage                  371        0.624579
wpt_name                      2        0.003367
